In [1]:
import numpy as np
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [2]:
actions = [
    'left',
    'right',
    'stop'
]

data = np.concatenate([
    np.load('dataset/seq_left_1655896673.npy'),
    np.load('dataset/seq_right_1655896673.npy'),
    np.load('dataset/seq_stop_1655896673.npy')
], axis=0)

data.shape

(2340, 30, 100)

In [3]:
x_data = data[:, :, :-1]
labels = data[:, 0, -1]

print(x_data.shape)
print(labels.shape)

(2340, 30, 99)
(2340,)


In [4]:
from tensorflow.keras.utils import to_categorical

y_data = to_categorical(labels, num_classes=len(actions))
y_data.shape

(2340, 3)

In [5]:
from sklearn.model_selection import train_test_split

x_data = x_data.astype(np.float32)
y_data = y_data.astype(np.float32)

x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.1, random_state=2021)

print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)

(2106, 30, 99) (2106, 3)
(234, 30, 99) (234, 3)


In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

model = Sequential([
    LSTM(64, activation='relu', input_shape=x_train.shape[1:3]),
    Dense(32, activation='relu'),
    Dense(len(actions), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 64)                41984     
                                                                 
 dense (Dense)               (None, 32)                2080      
                                                                 
 dense_1 (Dense)             (None, 3)                 99        
                                                                 
Total params: 44,163
Trainable params: 44,163
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

history = model.fit(
    x_train,
    y_train,
    validation_data=(x_val, y_val),
    epochs=200,
    callbacks=[
        ModelCheckpoint('models/model.h5', monitor='val_acc', verbose=1, save_best_only=True, mode='auto'),
        ReduceLROnPlateau(monitor='val_acc', factor=0.5, patience=50, verbose=1, mode='auto')
    ]
)

Epoch 1/200
66/66 [==============================] - ETA: 0s - loss: 65.5244 - acc: 0.3837
Epoch 1: val_acc improved from -inf to 0.52564, saving model to models\model.h5
66/66 [==============================] - 3s 27ms/step - loss: 65.5244 - acc: 0.3837 - val_loss: 14.6067 - val_acc: 0.5256 - lr: 0.0010
Epoch 2/200
64/66 [============================>.] - ETA: 0s - loss: 24.0809 - acc: 0.4756
Epoch 2: val_acc did not improve from 0.52564
66/66 [==============================] - 1s 20ms/step - loss: 24.0374 - acc: 0.4767 - val_loss: 12.1701 - val_acc: 0.5000 - lr: 0.0010
Epoch 3/200
65/66 [============================>.] - ETA: 0s - loss: 4.9500 - acc: 0.6120
Epoch 3: val_acc improved from 0.52564 to 0.63675, saving model to models\model.h5
66/66 [==============================] - 1s 22ms/step - loss: 4.9555 - acc: 0.6121 - val_loss: 3.8790 - val_acc: 0.6368 - lr: 0.0010
Epoch 4/200
63/66 [===========================>..] - ETA: 0s - loss: 3.0886 - acc: 0.6855
Epoch 4: val_acc improved 

In [ ]:
import matplotlib.pyplot as plt

fig, loss_ax = plt.subplots(figsize=(16, 10))
acc_ax = loss_ax.twinx()

loss_ax.plot(history.history['loss'], 'y', label='train loss')
loss_ax.plot(history.history['val_loss'], 'r', label='val loss')
loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
loss_ax.legend(loc='upper left')

acc_ax.plot(history.history['acc'], 'b', label='train acc')
acc_ax.plot(history.history['val_acc'], 'g', label='val acc')
acc_ax.set_ylabel('accuracy')
acc_ax.legend(loc='upper left')

plt.show()

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix
from tensorflow.keras.models import load_model

model = load_model('models/model.h5')

y_pred = model.predict(x_val)

multilabel_confusion_matrix(np.argmax(y_val, axis=1), np.argmax(y_pred, axis=1))